In [1]:
# https://pennylane.ai/qml/demos/tutorial_variational_classifier.html
%matplotlib inline
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
import matplotlib.pyplot as plt
from sklearn import datasets
import csv

In [2]:
dev = qml.device("default.qubit", wires=4)
#dev = qml.device("qiskit.aer", wires=4)

In [70]:
def get_angles(x):
    beta0 = 2 * np.arcsin(np.sqrt(x[1] ** 2) / np.sqrt(x[0] ** 2 + x[1] ** 2 + 1e-12))
    beta1 = 2 * np.arcsin(np.sqrt(x[3] ** 2) / np.sqrt(x[2] ** 2 + x[3] ** 2 + 1e-12))
    beta2 = 2 * np.arcsin(
        np.sqrt(x[2] ** 2 + x[3] ** 2)
        / np.sqrt(x[0] ** 2 + x[1] ** 2 + x[2] ** 2 + x[3] ** 2)
    )
    return np.array([beta2, -beta1 / 2, beta1 / 2, -beta0 / 2, beta0 / 2])


def amplitude_encoding(a):
    qml.RY(a[0], wires=0)

    qml.CNOT(wires=[0, 1])
    qml.RY(a[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[2], wires=1)

    qml.PauliX(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[3], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[4], wires=1)
    qml.PauliX(wires=0)
    
def rotation_encoding(features, evolution, entangle=False):
    # evolution should be a subclass of qml.operation.Operation
    assert(issubclass(evolution, qml.operation.Operation))
    # For Iris dataset, need 4 wires (4 features per data point)
    evolution(features[0], wires=0)
    evolution(features[1], wires=1)
    evolution(features[2], wires=2)
    evolution(features[3], wires=3)
    if entangle:
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[3, 0])
        
def full_rotation_encoding(features):
    for i in range(4):
        qml.RX(features[i], wires=i)
        qml.RY(features[i], wires=i)
        qml.RZ(features[i], wires=i)
        
def paper_rotation_encoding(features):
    # Based on the encoding from V. Havl´ıˇcek, A. D. C´orcoles, K. Temme, A. W. Harrow, A. Kandala, J. M. Chow, and
    # J. M. Gambetta. Supervised learning with quantum-enhanced feature spaces. Nature,
    # 567(7747):209–212, 2019. DOI: 10.1038/s41586-019-0980-2.
    # copied from "The power of quantum neural networks"
    for i in range(4):
        qml.Hadamard(wires=[i])
        
    for i in range(4):
        qml.RZ(features[i], wires=[i])
        
    for i in range(4):
        for j in range(i):
            qml.CNOT(wires=[j, i])
            angle = (np.pi - features[j]) * (np.pi - features[i])
            qml.RZ(angle, wires=[i])
            qml.CNOT(wires=[j, i])

In [4]:
x = np.array([0.53896774, 0.79503606, 0.27826503, 0.0], requires_grad=False)
ang = get_angles(x)

@qml.qnode(dev)
def test(angles):
    amplitude_encoding(ang)
    return qml.expval(qml.PauliZ(0))

test(ang)
print("x               : ", x)
print("angles          : ", ang)
print("amplitude vector: ", np.real(dev.state))

x               :  [0.53896774 0.79503606 0.27826503 0.        ]
angles          :  [ 0.56397465 -0.          0.         -0.97504604  0.97504604]
amplitude vector:  [ 5.38967743e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  7.95036065e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2.78265032e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -2.77555756e-17  0.00000000e+00  0.00000000e+00  0.00000000e+00]


In [5]:
#def layer(W):
#    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
#    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
#    qml.CNOT(wires=[0, 1])

def layer(W):
    # R(ϕ,θ,ω)=RZ(ω)RY(θ)RZ(ϕ)
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
    qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

In [6]:
def square_loss(labels, predictions):
    # https://pages.cs.wisc.edu/~matthewb/pages/notes/pdf/lossfunctions/SquaredLoss.pdf
    # l_squared(x, y, h) := (y - h(x))**2
    # L_s(h) := 1/(|S|) * sum(i=1 to |S|) of l_squared(x_i, y_i, h)
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def accuracy(labels, predictions):
    num_correct = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            num_correct += 1
    return num_correct / len(labels)

def cost(weights, bias, X, Y, encoding_method, encoding_args):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [7]:
@qml.qnode(dev)
def _circuit(weights, features, encoding_method, encoding_args, observable):
    encoding_method(features, *encoding_args)
    #statepreparation(angles)
    #evolution_encoding(features=angles, evolution=qml.RY, entangle=True)

    for W in weights:
        layer(W)

    return qml.expval(observable(0))


#def variational_classifier(weights, bias, angles):
#    return _circuit(weights, angles, encoding_method, encoding_args) + bias


def cost(weights, bias, features, labels):
    predictions = [variational_classifier(weights, bias, f) for f in features]
    return square_loss(labels, predictions)

In [8]:
'''
https://archive.ics.uci.edu/ml/datasets/iris
1. sepal length in cm
2. sepal width in cm
3. petal length in cm
4. petal width in cm
5. class:
-- Iris Setosa
-- Iris Versicolour
-- Iris Virginica
'''
iris = datasets.load_iris()
iris_features = np.array(iris.data)
iris_labels = np.array(iris.target)

# Index 100 is where type 2 starts
iris_features = iris_features[:100]
iris_labels = iris_labels[:100]

#data = np.loadtxt("data/iris_classes1and2_scaled.txt")
#X = data[:, 0:2]
X = iris_features
print("First X sample (original)  :", X[0])

# pad the vectors to size 2^2 with constant values
#padding = 0.3 * np.ones((len(X), 1))
#X_pad = np.c_[np.c_[X, padding], np.zeros((len(X), 1))]
#print("First X sample (padded)    :", X_pad[0])

# normalize each input
X_norm = np.array([subarray / np.linalg.norm(subarray) for subarray in iris_features])
#normalization = np.sqrt(np.sum(X ** 2, -1))
#X_norm = (X_pad.T / normalization).T
print("First X sample (normalized):", X_norm[0])

# angles for state preparation are new features
amp_features = np.array([get_angles(x) for x in X_norm], requires_grad=False)
print("First features sample      :", amp_features[0])

rot_features = 2 * np.pi * (X - np.min(X)) / (np.max(X) - np.min(X))
print(np.min(rot_features))
print(np.max(rot_features))

paper_features = 2 * (X - np.min(X)) / (np.max(X) - np.min(X)) - np.ones((100, 4))
print(np.min(paper_features))
print(np.max(paper_features))

test(amp_features[0])
print("x                          :", X_norm[0])
print("angles                     :", amp_features[0])
print("amplitude vector:          :", np.real(dev.state))

#Y = data[:, -1]
Y = iris_labels
# For now, switch to a 1/-1 result (1 = former 0, -1 = former 1)
Y = Y * -2 + 1
print(np.unique(Y, return_counts=True))

First X sample (original)  : [5.1 3.5 1.4 0.2]
First X sample (normalized): [0.80377277 0.55160877 0.22064351 0.0315205 ]
First features sample      : [ 0.44954297 -0.14189705  0.14189705 -0.60145471  0.60145471]
0.0
6.283185307179586
-1.0
1.0
x                          : [0.80377277 0.55160877 0.22064351 0.0315205 ]
angles                     : [ 0.44954297 -0.14189705  0.14189705 -0.60145471  0.60145471]
amplitude vector:          : [ 5.38967743e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  7.95036065e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2.78265032e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -2.77555756e-17  0.00000000e+00  0.00000000e+00  0.00000000e+00]
(tensor([-1,  1], requires_grad=True), array([50, 50], dtype=int64))


In [9]:
np.random.seed(0)
num_data = len(Y)
num_train = int(0.75 * num_data)
index = np.random.permutation(range(num_data))
amp_train = amp_features[index[:num_train]]
rot_train = rot_features[index[:num_train]]
paper_train = paper_features[index[:num_train]]
Y_train = Y[index[:num_train]]
amp_val = amp_features[index[num_train:]]
rot_val = rot_features[index[num_train:]]
paper_val = paper_features[index[num_train:]]
Y_val = Y[index[num_train:]]

# We need these later for plotting
X_train = X[index[:num_train]]
X_val = X[index[num_train:]]


In [10]:
num_qubits = 4
num_layers = 3

weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
#weights_init_4 = 0.01 * np.random.randn(num_layers, 4, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

In [11]:
class TestCase:
    def __init__(self, embedding, args, feats_train, feats_val, feats):
        self.embedding = embedding
        self.args = args
        self.feats_train = feats_train
        self.feats_val = feats_val
        self.feats = feats

In [71]:
#opt = NesterovMomentumOptimizer(0.01)
opt = qml.optimize.AdamOptimizer(stepsize=0.1)
batch_size = 5

# Prepare the tests
#embeddings = [amplitude_encoding, rotation_encoding, rotation_encoding, rotation_encoding,
#              rotation_encoding, rotation_encoding, rotation_encoding, paper_rotation_encoding]
#args = [[], [qml.RX], [qml.RX, True], [qml.RY], [qml.RY, True], [qml.RZ], [qml.RZ, True], []]
#features_train = [amp_train, rot_train, rot_train, rot_train, 
#                  rot_train, rot_train, rot_train, paper_train]
#features_val = [amp_val, rot_val, rot_val, rot_val, 
#                rot_val, rot_val, rot_val, paper_val]
#features = [amp_features, rot_features, rot_features, rot_features, 
#            rot_features, rot_features, rot_features, paper_features]
#wires_required = [2, 4, 4, 4, 4, 4, 4]
test1 = TestCase(amplitude_encoding, [], amp_train, amp_val, amp_features)
test2 = TestCase(rotation_encoding, [qml.RX], rot_train, rot_val, rot_features)
test3 = TestCase(rotation_encoding, [qml.RX, True], rot_train, rot_val, rot_features)
test4 = TestCase(rotation_encoding, [qml.RY], rot_train, rot_val, rot_features)
test5 = TestCase(rotation_encoding, [qml.RY, True], rot_train, rot_val, rot_features)
test6 = TestCase(rotation_encoding, [qml.RZ], rot_train, rot_val, rot_features)
test7 = TestCase(rotation_encoding, [qml.RZ, True], rot_train, rot_val, rot_features)
test8 = TestCase(paper_rotation_encoding, [], paper_train, paper_val, paper_features)
#test_cases = [test1, test2, test3, test4, test5, test6, test7, test8]
test9 = TestCase(full_rotation_encoding, [], rot_train, rot_val, rot_features)
test_cases = [test9]
observables = [qml.PauliX, qml.PauliY, qml.PauliZ]

# train the variational classifier
with open("Results.csv", "w", newline="") as output_csv:
    writer = csv.writer(output_csv)
    trial = 0
    header = ["Trial", "Observable", "Embedding", "Embedding Type", "Entangled", "Iteration", "Cost", "Train Acc", "Validation Acc"]
    writer.writerow(header)
    #for embed, arg, feats_train, feats_val, feats in zip(embeddings, args, features_train, features_val, features):
    for test_case in test_cases:
        embed_type = "N/A"
        if len(test_case.args) > 0:
            embed_type = test_case.args[0].__name__
        entangled = "N/A"
        if len(test_case.args) == 1:
            entangled = False
        elif len(test_case.args) == 2:
            entangled = True
        print("Using embedding", test_case.embedding.__name__, "with args", test_case.args, 
              "feats", test_case.feats_train[0])
        # Recreate the device with the required number of wires
        #dev = qml.device("default.qubit", wires=wire_count)
        print(dev.wires)

        #weights = weights_init
        #bias = bias_init
        # Run tests 3 times each
        for test in range(3):
            # Use a different observable each time (X, Y, Z)
            observable = observables[test]
            
            # Redefining variational_classifier for simplicity of cost function
            def variational_classifier(weights, bias, features):
                return _circuit(weights, features, test_case.embedding, test_case.args, observable) + bias
            
            trial += 1
            weights = weights_init
            bias = bias_init
            #print(qml.draw(_circuit)(weights, feats[0], embed, arg))
            for it in range(100): #changed from 60

                # Update the weights by one optimizer step
                batch_index = np.random.randint(0, num_train, (batch_size,))
                feats_train_batch = test_case.feats_train[batch_index]
                Y_train_batch = Y_train[batch_index]
                weights, bias, _, _ = opt.step(cost, weights, bias, feats_train_batch, Y_train_batch)

                # Compute predictions on train and validation set
                predictions_train = [np.sign(variational_classifier(weights, bias, f)) for f in test_case.feats_train]
                predictions_val = [np.sign(variational_classifier(weights, bias, f)) for f in test_case.feats_val]

                # Compute accuracy on train and validation set
                acc_train = accuracy(Y_train, predictions_train)
                acc_val = accuracy(Y_val, predictions_val)

                temp_cost = cost(weights, bias, test_case.feats, Y)
                print(
                    "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
                    "".format(it + 1, temp_cost, acc_train, acc_val)
                )
                writer.writerow([trial, observables[test].__name__, test_case.embedding.__name__, 
                                 embed_type, entangled, it + 1, temp_cost, acc_train, acc_val])
                #if acc_train == acc_val and acc_val == 1:
                #    break

Using embedding full_rotation_encoding with args [] feats [4.46197217 3.00500167 1.36590985 0.27318197]
<Wires = [0, 1, 2, 3]>
Iter:     1 | Cost: 0.8612203 | Acc train: 0.7333333 | Acc validation: 0.7600000 
Iter:     2 | Cost: 0.7562700 | Acc train: 0.7866667 | Acc validation: 0.8000000 
Iter:     3 | Cost: 0.6873499 | Acc train: 0.8266667 | Acc validation: 0.8000000 
Iter:     4 | Cost: 0.6336239 | Acc train: 0.8400000 | Acc validation: 0.8400000 
Iter:     5 | Cost: 0.6215822 | Acc train: 0.8133333 | Acc validation: 0.8000000 
Iter:     6 | Cost: 0.5785733 | Acc train: 0.8666667 | Acc validation: 0.8400000 
Iter:     7 | Cost: 0.5317883 | Acc train: 0.9066667 | Acc validation: 0.9200000 
Iter:     8 | Cost: 0.5115499 | Acc train: 0.9066667 | Acc validation: 0.9200000 
Iter:     9 | Cost: 0.4871930 | Acc train: 0.9200000 | Acc validation: 0.9200000 
Iter:    10 | Cost: 0.4852844 | Acc train: 0.9066667 | Acc validation: 0.9200000 
Iter:    11 | Cost: 0.4729119 | Acc train: 0.9066667 

Iter:   100 | Cost: 0.1057669 | Acc train: 1.0000000 | Acc validation: 1.0000000 
Iter:     1 | Cost: 0.9325143 | Acc train: 0.8000000 | Acc validation: 0.9200000 
Iter:     2 | Cost: 0.8588151 | Acc train: 0.8000000 | Acc validation: 0.9200000 
Iter:     3 | Cost: 0.7289320 | Acc train: 0.8133333 | Acc validation: 0.9200000 
Iter:     4 | Cost: 0.6064874 | Acc train: 0.8133333 | Acc validation: 0.8800000 
Iter:     5 | Cost: 0.4791980 | Acc train: 0.8800000 | Acc validation: 0.9600000 
Iter:     6 | Cost: 0.3949027 | Acc train: 0.8933333 | Acc validation: 1.0000000 
Iter:     7 | Cost: 0.3601405 | Acc train: 0.9066667 | Acc validation: 0.9600000 
Iter:     8 | Cost: 0.3502094 | Acc train: 0.9600000 | Acc validation: 0.9600000 
Iter:     9 | Cost: 0.3966039 | Acc train: 0.9466667 | Acc validation: 0.9600000 
Iter:    10 | Cost: 0.4019608 | Acc train: 0.9733333 | Acc validation: 0.9600000 
Iter:    11 | Cost: 0.3407705 | Acc train: 1.0000000 | Acc validation: 1.0000000 
Iter:    12 | Co

Iter:   100 | Cost: 0.1483593 | Acc train: 1.0000000 | Acc validation: 1.0000000 
Iter:     1 | Cost: 1.2005068 | Acc train: 0.5333333 | Acc validation: 0.4400000 
Iter:     2 | Cost: 1.0473781 | Acc train: 0.5733333 | Acc validation: 0.4800000 
Iter:     3 | Cost: 0.8295782 | Acc train: 0.6266667 | Acc validation: 0.6000000 
Iter:     4 | Cost: 0.6335132 | Acc train: 0.7600000 | Acc validation: 0.7600000 
Iter:     5 | Cost: 0.5615995 | Acc train: 0.8666667 | Acc validation: 0.8800000 
Iter:     6 | Cost: 0.6044426 | Acc train: 0.8000000 | Acc validation: 0.8400000 
Iter:     7 | Cost: 0.6464675 | Acc train: 0.7333333 | Acc validation: 0.8400000 
Iter:     8 | Cost: 0.6084054 | Acc train: 0.7600000 | Acc validation: 0.8400000 
Iter:     9 | Cost: 0.5371937 | Acc train: 0.7733333 | Acc validation: 0.8400000 
Iter:    10 | Cost: 0.4069923 | Acc train: 0.8666667 | Acc validation: 1.0000000 
Iter:    11 | Cost: 0.3198920 | Acc train: 0.9066667 | Acc validation: 1.0000000 
Iter:    12 | Co

Iter:   100 | Cost: 0.1150874 | Acc train: 1.0000000 | Acc validation: 1.0000000 


In [14]:
#_circuit(weights, paper_features[0], paper_rotation_encoding, [])
#dev._circuit.draw(output="mpl")

In [69]:
dev2 = qml.device("default.qubit", wires=6)

def layer_temp(W):
    # R(ϕ,θ,ω)=RZ(ω)RY(θ)RZ(ϕ)
    qml.Rot(W[0], W[1], W[2], wires=0)
    qml.Rot(W[3], W[4], W[5], wires=1)
    qml.Rot(W[6], W[7], W[8], wires=2)
    qml.Rot(W[9], W[10], W[11], wires=3)

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

'''
test1 = TestCase(amplitude_encoding, [], amp_train, amp_val, amp_features)
test2 = TestCase(rotation_encoding, [qml.RX], rot_train, rot_val, rot_features)
test3 = TestCase(rotation_encoding, [qml.RX, True], rot_train, rot_val, rot_features)
test4 = TestCase(rotation_encoding, [qml.RY], rot_train, rot_val, rot_features)
test5 = TestCase(rotation_encoding, [qml.RY, True], rot_train, rot_val, rot_features)
test6 = TestCase(rotation_encoding, [qml.RZ], rot_train, rot_val, rot_features)
test7 = TestCase(rotation_encoding, [qml.RZ, True], rot_train, rot_val, rot_features)
test8 = TestCase(paper_rotation_encoding, [], paper_train, paper_val, paper_features
'''    
    
@qml.qnode(dev2)
def circuit_temp(weights, features):
    rotation_encoding(features, qml.RX, True)
    #amplitude_encoding(features)
    #paper_rotation_encoding(features)
    #statepreparation(angles)
    #evolution_encoding(features=angles, evolution=qml.RY, entangle=True)

    #for W in weights:
    layer_temp(weights)

    return qml.expval(qml.PauliY(0))

weights_temp = 0.01 * np.random.randn(12, requires_grad=True)
weights_temp = np.array(weights_temp, requires_grad=True)

mt_fn = qml.metric_tensor(circuit_temp)
# test1 = TestCase(amplitude_encoding, [], amp_train, amp_val, amp_features)
#print(weights_temp, rot_features[0])
metric_tensor = mt_fn(weights_temp, amp_features[0])
#print(metric_tensor)
#print(amp_features[0])
#print(rot_features[0])
eigvals, eigvecs = np.linalg.eigh(metric_tensor)
print(eigvals)

[2.01084158e-09 2.43220920e-08 1.33579591e-05 1.83344619e-05
 4.75412096e-03 1.72620960e-02 1.79350592e-01 2.37104307e-01
 2.48599231e-01 2.50000685e-01 2.62825698e-01 4.21584966e-01]
